In [1]:
import pandas as pd

In [3]:
selloutData = \
    pd.read_csv("../dataset/kopo_channel_seasonality_new.csv")

[데이터타입 통합]
kopo_channel_seasonality_new.csv 파일을 불러온 후
selloutData 변수에 담으세요 이후 
QTY컬럼→실수(float), 이외컬럼→문자(str)로 변경하세요

### A.데이터 전처리

### A-1. 타입 통합 및 불량데이터 처리

In [17]:
allColumns = list(selloutData.columns)

In [18]:
for i in range(0, len(allColumns)):
    selloutData[ allColumns[i] ] = selloutData[ allColumns[i] ].astype(str)

In [19]:
selloutData.QTY = selloutData.QTY.astype(float)

[불량 데이터 처리]
kopo_channel_seasonality_new.csv 자료를 담은 
selloutData 변수에서
QTY컬럼 음수(반품)인 경우 0, 양수인 경우 기존 QTY 값유지하는 로직을 적용하여 QTY_NEW 컬럼을 추가하세요


In [23]:
selloutData.loc[selloutData.QTY < 0,"NEW_QTY" ] = 0
selloutData.loc[selloutData.QTY >= 0,"NEW_QTY" ] = selloutData.QTY

In [29]:
selloutData.loc[ (selloutData.QTY < 0) & 
                 (selloutData.NEW_QTY != 0) ]

,REGIONID,PRODUCT,YEARWEEK,QTY,NEW_QTY


In [30]:
selloutData.loc[ (selloutData.QTY > 0) & 
                 (selloutData.NEW_QTY != selloutData.QTY) ]

,REGIONID,PRODUCT,YEARWEEK,QTY,NEW_QTY


[데이터 통합]
selloutData 자료에서 
YEAR, WEEK 컬럼을 생성하고53주차를 제거한다!!!


In [33]:
selloutData["YEAR"] = selloutData["YEARWEEK"].str[0:4]
selloutData["WEEK"] = selloutData["YEARWEEK"].str[4:6]

In [157]:
refinedSelloutData = selloutData.loc[selloutData.WEEK!=53]

In [148]:
sortKey = ["REGIONID","PRODUCT","YEARWEEK"]

In [149]:
sortedData = selloutData.sort_values(sortKey)

In [150]:
groupKey = ["REGIONID","PRODUCT","YEAR"]

In [155]:
groupData = \
    sortedData.groupby(groupKey)[["QTY"]].agg("mean").reset_index()

In [162]:
groupData = groupData.rename(columns={"QTY":"QTY_MEAN"})

In [163]:
joinKey = ["REGIONID","PRODUCT","YEAR"]

In [166]:
mergedData = pd.merge(left=refinedSelloutData,
         right=groupData,
         how="left",
         on = joinKey)

In [170]:
finalData = mergedData.sort_values(by=sortKey)
finalData["SEASONALITY"] = finalData["QTY"] / finalData["QTY_MEAN"]
finalData

,REGIONID,PRODUCT,YEARWEEK,QTY,NEW_QTY,YEAR,WEEK,QTY_MEAN,SEASONALITY
298,A00,PRODUCT34,201401,661.0,661.0,2014,01,275.961538,2.395261
1757,A00,PRODUCT34,201402,679.0,679.0,2014,02,275.961538,2.460488
3125,A00,PRODUCT34,201403,578.0,578.0,2014,03,275.961538,2.094495
205,A00,PRODUCT34,201404,532.0,532.0,2014,04,275.961538,1.927805
4369,A00,PRODUCT34,201405,516.0,516.0,2014,05,275.961538,1.869826
...,...,...,...,...,...,...,...,...,...
102292,A77,PRODUCT12,201648,4152.0,4152.0,2016,48,4837.153846,0.858356
97525,A77,PRODUCT12,201649,5086.0,5086.0,2016,49,4837.153846,1.051445
102329,A77,PRODUCT12,201650,5846.0,5846.0,2016,50,4837.153846,1.208562
101211,A77,PRODUCT12,201651,4933.0,4933.0,2016,51,4837.153846,1.019815


In [171]:
groupKey2 = ["REGIONID","PRODUCT","WEEK"]

In [175]:
seaonData = finalData.groupby(groupKey2)["SEASONALITY"].agg(["mean"]).reset_index()
seaonData

,REGIONID,PRODUCT,WEEK,mean
0,A00,PRODUCT34,01,1.577131
1,A00,PRODUCT34,02,1.763571
2,A00,PRODUCT34,03,1.323958
3,A00,PRODUCT34,04,1.496774
4,A00,PRODUCT34,05,1.064810
...,...,...,...,...
42077,A77,PRODUCT12,49,1.093556
42078,A77,PRODUCT12,50,1.385585
42079,A77,PRODUCT12,51,1.254739
42080,A77,PRODUCT12,52,1.387482


# 1. 데이터 이후

In [38]:
refinedData = \
    selloutData.loc[selloutData.WEEK != exceptWeek ]

In [77]:
groupKey = ["REGIONID","PRODUCT"]

In [82]:
groupData = selloutData.groupby(groupKey)["QTY"].agg(["mean"]).reset_index()

In [86]:
groupData2 = groupData.rename(columns={"mean":"QTY_MEAN"})

In [87]:
groupData2

,REGIONID,PRODUCT,QTY_MEAN
0,A00,PRODUCT34,132.445860
1,A00,PRODUCT58,105.235669
2,A00,PRODUCT59,36086.547771
3,A01,PRODUCT1,5716.726115
4,A01,PRODUCT12,197052.777070
...,...,...,...
789,A70,PRODUCT34,2165.585987
790,A70,PRODUCT58,61.496815
791,A70,PRODUCT59,69471.617834
792,A77,PRODUCT1,2890.745223


# 조인

In [90]:
joinMaster = \
    pd.read_csv("../dataset/join_region_master.csv", encoding="ms949")

In [91]:
joinSellout = \
    pd.read_csv("../dataset/join_kopo_product_volume.csv")

In [98]:
joinSellout

,REGIONID,PRODUCT,YEARWEEK,QTY
0,A01,ST0001,201901,3000
1,A01,ST0001,201902,3500
2,A02,ST0002,201901,4000
3,A02,ST0003,201902,7000


In [99]:
joinKey = ["REGIONID"]

In [106]:
mergedData = pd.merge(left=joinMaster, #기준정보 
         right=joinSellout, # 실적정보
         how = "inner", # innerjoin)
         on = joinKey)
mergedData

,REGIONID,REGIONNAME,PRODUCT,YEARWEEK,QTY
0,A01,한국,ST0001,201901,3000
1,A01,한국,ST0001,201902,3500
2,A02,미국,ST0002,201901,4000
3,A02,미국,ST0003,201902,7000


In [111]:
joinMaster2 = joinMaster.loc[joinMaster.REGIONID=="A01"]

In [112]:
joinMaster2

,REGIONID,REGIONNAME
0,A01,한국


In [120]:
row1 = ["A99","ST0010","201901",5000]

columns = joinSellout.columns

chugadata = pd.DataFrame([ row1 ], columns=columns )

joinSellout2 = joinSellout.append(chugadata)

In [135]:
joinSellout3 = joinSellout2.rename(columns={"REGIONID":"AP2ID"})

In [138]:
# joinMaster

In [136]:
joinSellout3

,AP2ID,PRODUCT,YEARWEEK,QTY
0,A01,ST0001,201901,3000
1,A01,ST0001,201902,3500
2,A02,ST0002,201901,4000
3,A02,ST0003,201902,7000
0,A99,ST0010,201901,5000


In [141]:
a01Data = joinSellout.loc[joinSellout.REGIONID=="A01"]
a02Data = joinSellout.loc[joinSellout.REGIONID=="A02"]

In [144]:
pd.concat([a01Data,a02Data], axis=0)

,REGIONID,PRODUCT,YEARWEEK,QTY
0,A01,ST0001,201901,3000
1,A01,ST0001,201902,3500
2,A02,ST0002,201901,4000
3,A02,ST0003,201902,7000


In [143]:
a01Data.append(a02Data)

,REGIONID,PRODUCT,YEARWEEK,QTY
0,A01,ST0001,201901,3000
1,A01,ST0001,201902,3500
2,A02,ST0002,201901,4000
3,A02,ST0003,201902,7000


In [139]:
pd.merge(left=joinSellout3, 
         right=joinMaster, 
         how="left", 
         left_on=["AP2ID"],right_on=["REGIONID"])

,AP2ID,PRODUCT,YEARWEEK,QTY,REGIONID,REGIONNAME
0,A01,ST0001,201901,3000,A01,한국
1,A01,ST0001,201902,3500,A01,한국
2,A02,ST0002,201901,4000,A02,미국
3,A02,ST0003,201902,7000,A02,미국
4,A99,ST0010,201901,5000,NaN,NaN


In [133]:
pd.merge(left=joinSellout2, right=joinMaster, on=joinKey, how="outer")

,REGIONID,PRODUCT,YEARWEEK,QTY,REGIONNAME
0,A01,ST0001,201901,3000.0,한국
1,A01,ST0001,201902,3500.0,한국
2,A02,ST0002,201901,4000.0,미국
3,A02,ST0003,201902,7000.0,미국
4,A99,ST0010,201901,5000.0,NaN
5,A03,NaN,NaN,NaN,프랑스
6,A04,NaN,NaN,NaN,독일


In [117]:
joinMaster

,REGIONID,REGIONNAME
0,A01,한국
1,A02,미국
2,A03,프랑스
3,A04,독일


In [116]:
joinSellout

,REGIONID,PRODUCT,YEARWEEK,QTY
0,A01,ST0001,201901,3000
1,A01,ST0001,201902,3500
2,A02,ST0002,201901,4000
3,A02,ST0003,201902,7000


In [115]:
pd.merge(left=joinSellout,
         right=joinMaster2,
         how="left",
         on=joinKey)

,REGIONID,PRODUCT,YEARWEEK,QTY,REGIONNAME
0,A01,ST0001,201901,3000,한국
1,A01,ST0001,201902,3500,한국
2,A02,ST0002,201901,4000,NaN
3,A02,ST0003,201902,7000,NaN


In [92]:
joinSellout

,REGIONID,PRODUCT,YEARWEEK,QTY
0,A01,ST0001,201901,3000
1,A01,ST0001,201902,3500
2,A02,ST0002,201901,4000
3,A02,ST0003,201902,7000


In [40]:
refinedData.shape

(123864, 7)

In [39]:
selloutData.shape

(124658, 7)

In [51]:
groupKey = ["PRODUCT"]

In [64]:
columns = ["이름","성별","과목","점수"]

groupDf = pd.DataFrame( [["김효관","남","수학",50],
 ["김어진","남","수학",80],
 ["안민정","여","수학",90],
 ["안정훈","여","국어",120],
 ["김여진","여","수학",95],
 ["이동연","남","국어",80]   ] , columns =columns )

In [66]:
groupKey = ["과목"]

In [71]:
testDf = pd.read_csv("../dataset/kopo_customerdata.csv")

In [73]:
groupKey = ["GENDER"]

In [75]:
testDf.groupby(by=groupKey)["TOTAL_AMOUNT"].agg(["count"]).reset_index()

,GENDER,count
0,Female,36754
1,Male,263064
2,Not Available,182


In [69]:
groupDf.groupby(by=groupKey)["점수"].agg(["count"]).reset_index()

,과목,count
0,국어,2
1,수학,4


In [59]:
groupKey = ["과목"]

In [63]:
testDf

,이름,성별,과목,점수
0,김효관,남,수학,50
1,김어진,남,수학,80
2,안민정,여,수학,90
3,안정훈,여,국어,120
4,김여진,여,수학,95
5,이동연,남,국어,80


In [62]:
testDf.groupby(by=groupKey)["점수"].agg(["mean"]).reset_index()

,과목,mean
0,국어,100.00
1,수학,78.75
